# Import Library

In [ ]:
import numpy as np
import h5py as h5
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Extracting the train dataset from the h5 file.
f = h5.File("../input/snake-classification-train/train_set.hdf5",'r')

print("Stored data:",list(f.keys()))

# Storing the original data "permanently".
images_train_orig = f["images_train"].value
labels_train_orig = f["labels_train"].value
f.close()

# # Extracting the dev dataset from the h5 file.
# f = h5.File("./dataset/dev_set.hdf5",'r')
# print("Stored data:",list(f.keys()))
# # Storing the original data "permanently".
# images_dev_orig = f["images_dev"].value
# labels_dev_orig = f["labels_dev"].value
# f.close()

In [ ]:
images_train_orig[1]

In [ ]:
labels_train_orig

In [ ]:
print("Train set")
print("Images shape:", images_train_orig.shape)
print("Images dtype:", images_train_orig.dtype)
print("Min, max and mean pixel values:", images_train_orig.min(),"-",
      images_train_orig.max(),"-","{:5.1f}".format(images_train_orig.mean()))
print("Labels shape:", labels_train_orig.shape)
print("Labels dtype:", labels_train_orig.dtype)

In [ ]:
def species(i):
    if i == 1:
        name = "python"
    elif i==0:
        name = "rattlesnake"
    else:
        name = "dunno"
    return name

In [ ]:
i = 7
print("Label =",labels_train_orig[i],"- A wild", species(labels_train_orig[i]), "appears.")
plt.imshow(images_train_orig[i])
plt.show()

# Preparing the data
As we saw earlyer, images are stored using 0-255 pixel values. We will scale them to 0-1 float values. Also, we linearize the pixel values on width, height and channels for each image.

In [ ]:
# Copying the original data for manipulation.
images_train = images_train_orig
labels_train = labels_train_orig.reshape([1,len(labels_train_orig)])
 

In [ ]:
# Scaling the data
images_train = images_train/255
images_train = images_train.reshape((len(images_train),-1))
images_train = images_train.transpose()

In [ ]:
# Shapes
imsize = images_train.shape[0]
print("Images train shape")
print(images_train.shape)
print(labels_train.shape)

# Logistic Regression
Logistic regression (LR) is one of the simplest classification methods available, and shares important key ideas with shallow and deep neural networks, which we will use later. Although it's arguable that we can call LR a neural learning procedure, we will consider it so for the sake of this project's consistency. From now on, we will build all models' core engines using TensorFlow.

# Building the model
Now that our data is ready, let's build the model in tensorflow.

In [ ]:
tf.__version__

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
# Create placeholder for images, call it X.
tf.compat.v1.reset_default_graph()
X = tf.compat.v1.placeholder(tf.float32, shape=(imsize,None))
Y = tf.compat.v1.placeholder(tf.float32,shape=(1,None))

# Create weights and biases. Initialize to zero.
W = tf.compat.v1.get_variable("W", [1,imsize], initializer=tf.zeros_initializer())
b = tf.compat.v1.get_variable("b", [1],initializer=tf.zeros_initializer())

# Make the affine transformation
logits = tf.add(tf.matmul(W,X),b)

# Calculating the cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y,logits=logits))

## Training the model
Training a logistic regression is pretty straightforward as demonstrated below. We use gradient descent optimizer, considering the gradient on the mean logloss over the whole trainning dataset. We also compute and store the cost on the dev dataset to analize the out-of-sample error.

In [ ]:
epochs = 500
alpha = 0.001
# Creating the optimizer

optimizer = tf.compat.v1.train.GradientDescentOptimizer(alpha).minimize(cost)

# Initializing global variables
init = tf.compat.v1.global_variables_initializer()

# Running session
with tf.compat.v1.Session() as sess:
    sess.run(init)
    costs_train = []
#     costs_dev = []
    for epoch in range(epochs):
        _,epoch_cost = sess.run([optimizer,cost],feed_dict={X:images_train,Y:labels_train})
        costs_train.append(epoch_cost)
#         cost_dev = sess.run(cost,feed_dict={X:images_dev,Y:labels_dev})
        print(epoch,epoch_cost)
#         print(epoch,epoch_cost, cost_dev)
#         costs_dev.append(cost_dev)
    Weights, Bias = sess.run([W,b])

# Error curves
We plot the train and dev set error stored during training to view the behavior of the optimizaton scheme.

In [ ]:
train_curve, = plt.plot(costs_train, label = 'Train error')
# test_curve,  = plt.plot(costs_dev, label = 'Dev error')
# plt.legend(handles=[train_curve,test_curve])
plt.legend(handles=[train_curve])
plt.xlabel('Epochs')
plt.ylabel('Mean log loss')

# Model accuracy

In [ ]:
def predict(x,w,b):
    z = np.dot(w,x)+b
    probs = 1/(1+np.exp(-z))
    y_pred = (probs > 0.5).astype(int)
    return y_pred

In [ ]:
def accuracy(label,y_pred):
    acc = np.mean(np.equal(label,y_pred))
    return acc

In [ ]:
y_train_pred = predict(images_train,Weights,Bias)

acc = accuracy(labels_train,y_train_pred)

print("Train set accuracy:", acc)

# y_dev_pred = predict(images_dev,Weights,Bias)

# acc = accuracy(labels_dev,y_dev_pred)

# print("Dev set accuracy:",acc)

# Prediction

In [ ]:
i = 0
sample = images_train[:,i]
target = labels_train[0,i]
prediction = predict(sample,Weights,Bias)
print("It says it's a {}, it's actually a {}".format(species(prediction),species(target)))
print(("Good guess model! Take a cookie." if target == prediction else "Bad model! No cookie for you."))
plt.imshow(images_train_orig[i])
plt.show()

In [ ]:
def predictions(i):
    sample = images_train[:,i]
    target = labels_train[0,i]
    prediction = predict(sample,Weights,Bias)
    print("It says it's a {}, it's actually a {}".format(species(prediction),species(target)))
    print(("Good guess model! Take a cookie." if target == prediction else "Bad model! No cookie for you."))
    plt.imshow(images_train_orig[i])
    plt.show()

In [ ]:
predictions(1)

In [ ]:
predictions(3)